In [2]:
import geopandas as gpd

In [4]:
h_gdf = gpd.read_parquet('../data/houses_price_demo.parquet').iloc[:, :3]
h_gdf.to_crs(h_gdf.estimate_utm_crs().to_epsg(), inplace=True)
h_gdf.head(2)

,avg_m2_price_rent,mean_price_rent,geometry
0,724.346716,45426.664221,"MULTIPOLYGON (((317501.141 6657001.314, 317490..."
1,883.362808,45554.482535,"POLYGON ((347889.496 6647694.723, 347900.553 6..."


In [5]:
services = ["clinic", "hospitals", "kindergarten", "mfc", "schools"]

for service in services:
    gdf = gpd.read_parquet(
        f"../provisions_data/spb/{service}.parquet"
    )
    gdf.geometry = gdf.representative_point()
    h_gdf = (
        h_gdf.sjoin(
            gdf[["total_provision_assessment", "geometry"]],
            predicate="intersects",
            how="inner",
        )
        .rename(columns={"total_provision_assessment": f"P_{service}"})
        .reset_index(drop=True)
        .drop(columns=['index_right'])
    )

In [6]:
h_gdf.head(2)

,avg_m2_price_rent,mean_price_rent,geometry,P_clinic,P_hospitals,P_kindergarten,P_mfc,P_schools
0,724.346716,45426.664221,"MULTIPOLYGON (((317501.141 6657001.314, 317490...",1,0,0.0,1.0,0.0
1,883.362808,45554.482535,"POLYGON ((347889.496 6647694.723, 347900.553 6...",1,1,0.0,1.0,1.0


In [7]:
h_gdf.to_parquet('../data/houses_price_demo.parquet')